Hey, hope you're doing well. This project will analyze EV charging utilization metrics from a public Kaggle dataset which measures EV charging station usage from Palo Alto.

To begin, let's first upload the necessary libraries:

In [6]:
import zipfile

from pathlib import Path

import pandas as pd

Let's now unzip this dataset locally here (it is a zip file as it is an extremely large dataset).

In [7]:
zip_path = Path("data/EVChargingStationUsage.csv.zip")

extract_dir = Path("data/extracted") # Placing the unzipped CSV in a temporary folder

# Let's now unzip this file:

with zipfile.ZipFile(zip_path, "r") as z: # To open the zip to place it in extract_dir
    z.extractall(extract_dir)

list(extract_dir.glob("*"))

# Let's check if that extraction worked.    

[PosixPath('data/extracted/EVChargingStationUsage.csv'),
 PosixPath('data/extracted/.ipynb_checkpoints')]

Because this dataset tracks usages, it is an event-based and relational system. In practice, each usage belongs to exactly one station, and each station relates to multiple usages. We can create relational tables for this, and we will use SQL to do so.

In [8]:
import sqlite3

db_path = Path("data/ev_charging.db") # to create a file path object
conn = sqlite3.connect(db_path) # now let's connect to the database
print(db_path.resolve()) # let's see where our db lives

conn.execute("SELECT 1").fetchone() # as a test

/home/jupyter/ev-charging-station-analysis/ev-charging-stations-analysis/data/ev_charging.db


(1,)

So now our databse exists, and the connection seems to work- but our database is empty, so we need to put the raw CSV data as a "raw table".

In [14]:
# Let's first just see how big our data is:

df = pd.read_csv("data/extracted/EVChargingStationUsage.csv",
                low_memory = False)

df.shape

(259415, 33)

In [15]:
# Now, let's write this into SQLite

df.to_sql(
    "raw_sessions", # we'll choose this as the table name
    conn, if_exists = "replace",
    index = False
)

# Let's confirm this table exists

pd.read_sql_query(
    "SELECT * FROM raw_sessions LIMIT 5;", # to test a small query
    conn
)

,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),...,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284,None,None,None
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,-122.160309,USD,0.0,Customer,4,94301.0,4169,None,None,None
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169,None,None,None
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,-122.160309,USD,0.0,Customer,6,94302.0,2545,None,None,None
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765,None,None,None


In [18]:
# We can quickly confirm our table and row count:

pd.read_sql_query(
    "SELECT COUNT(*) AS n_rows FROM raw_sessions;", conn)

,n_rows
0,259415


In [19]:
# It matches. Let's now just confirm all our column names:

cols = pd.read_sql_query("PRAGMA table_info(raw_sessions);", conn)

cols

,cid,name,type,notnull,dflt_value,pk
0,0,Station Name,TEXT,0,None,0
1,1,MAC Address,TEXT,0,None,0
2,2,Org Name,TEXT,0,None,0
3,3,Start Date,TEXT,0,None,0
4,4,Start Time Zone,TEXT,0,None,0
5,5,End Date,TEXT,0,None,0
6,6,End Time Zone,TEXT,0,None,0
7,7,Transaction Date (Pacific Time),TEXT,0,None,0
8,8,Total Duration (hh:mm:ss),TEXT,0,None,0
9,9,Charging Time (hh:mm:ss),TEXT,0,None,0


Looking at this, we can get a better sense of what we need to do next. Let's focus on creating tables for stations (one row per station), users (one row per user), and sessions (one row per charging session, which references the station and user). Before we do that though, let's make sure we these variables have enough non-null values:

In [29]:
pd.read_sql_query("""
SELECT
COUNT(*) AS rows,
COUNT(DISTINCT "EVSE ID") AS distinct_stations,
SUM(CASE WHEN "EVSE ID" IS NULL THEN 1 ELSE 0 END) AS null_station_id
FROM raw_sessions;
""", conn)



,rows,distinct_stations,null_station_id
0,259415,51,78948


In [30]:
pd.read_sql_query("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT "Plug In Event Id") AS distinct_events,
  SUM(CASE WHEN "Plug In Event Id" IS NULL THEN 1 ELSE 0 END) AS null_event_id
FROM raw_sessions;
""", conn)


,rows,distinct_events,null_event_id
0,259415,36838,0


In [31]:
pd.read_sql_query("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT "User ID") AS distinct_users,
  SUM(CASE WHEN "User ID" IS NULL OR TRIM("User ID")='' THEN 1 ELSE 0 END) AS null_user_id
FROM raw_sessions;
""", conn)

,rows,distinct_users,null_user_id
0,259415,21441,7677


,n_stations
0,51
